In [6]:
from sklearn.datasets import fetch_openml

In [7]:
mnist = fetch_openml('mnist_784')

In [8]:
x , y = mnist['data'], mnist['target']

In [9]:
y[0]

'5'

In [10]:
from keras.utils import to_categorical
y = to_categorical(y)

Using TensorFlow backend.
/home/aryan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/aryan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/aryan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/aryan/.local/lib/python3.6/site-packages/tensorflow/python/fra

In [11]:
import numpy as np 

In [12]:
print(y)

[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [13]:
x.shape

(70000, 784)

In [14]:
def sigmoid(x):
    return 1 / (1 + np.e ** -x)

In [33]:
activation_function = sigmoid
from scipy.stats import truncnorm
def truncated_normal(mean=0, sd=1, low=0, upp=10):
    return truncnorm((low - mean) / sd, 
                     (upp - mean) / sd, 
                     loc=mean, 
                     scale=sd)
class NeuralNetwork:
    
    def __init__(self,innode,outnode,hidnode,learning_rate):
        self.innode = innode
        self.outnode= outnode
        self.hidnode = hidnode
        self.learning_rate = learning_rate 
        self.create_weight_matrices()
        
    def create_weight_matrices(self):
        rad = 1 / np.sqrt(self.innode)
        X = truncated_normal(mean=0,sd=1,low=-rad,upp=rad)
        self.wih = X.rvs((self.hidnode,self.innode))
        rad = 1 / np.sqrt(self.hidnode)
        X = truncated_normal(mean=0, sd=1, low=-rad, upp=rad)
        self.who = X.rvs((self.outnode,self.hidnode))
        
    
    def train(self, input_vector, target_vector):
        input_vector = input_vector/255.0
        input_vector = np.array(input_vector, ndmin=2).T
        target_vector = np.array(target_vector, ndmin=2).T
        
        output_vector1 = np.dot(self.wih,input_vector)
        output_hidden = activation_function(output_vector1)
        
        output_vector2 = np.dot(self.who,output_hidden)
        output_network = activation_function(output_vector2)
        
        output_errors = target_vector - output_network
        tmp = output_errors * output_network \
              * (1.0 - output_network)     
        tmp = self.learning_rate  * np.dot(tmp,output_hidden.T)
        self.who += tmp
        hidden_errors = np.dot(self.who.T,output_errors)
        tmp = hidden_errors * output_hidden * (1.0 - output_hidden)
        self.wih += self.learning_rate * np.dot(tmp, input_vector.T)
        
        
    
    def run(self, input_vector):
        input_vector = np.array(input_vector, ndmin=2).T
        output_vector = np.dot(self.wih,input_vector)
        output_vector = activation_function(output_vector)
        
        output_vector = np.dot(self.who,output_vector)
        output_vector = activation_function(output_vector)
    
        return output_vector
            
    def confusion_matrix(self, data_array, labels):
        cm = np.zeros((10, 10), int)
        for i in range(len(data_array)):
            res = self.run(data_array[i])
            res_max = res.argmax()
            target = labels[i][0]
            cm[res_max, int(target)] += 1
        return cm    
    def precision(self, label, confusion_matrix):
        col = confusion_matrix[:, label]
        return confusion_matrix[label, label] / col.sum()
    
    def recall(self, label, confusion_matrix):
        row = confusion_matrix[label, :]
        return confusion_matrix[label, label] / row.sum()
        
    
    def evaluate(self, data, labels):
        corrects, wrongs = 0, 0
        for i in range(len(data)):
            res = self.run(data[i])
            res_max = res.argmax()
            if res_max == labels[i]:
                corrects += 1
            else:
                wrongs += 1
        return corrects, wrongs
            

In [34]:
activation_function(2)

0.8807970779778823

In [35]:


ANN = NeuralNetwork(innode = 784, 
                    outnode = 10, 
                    hidnode = 100,
                    learning_rate = 0.1)
    
    
for i in range(len(x)):
    ANN.train(x[i], y[i])


In [39]:


for i in range(20):
    res = ANN.run(x[i])
    print(y[i],np.argmax(res))
    


[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.] 5
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 0
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.] 4
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.] 1
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.] 9
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.] 2
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.] 1
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.] 3
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.] 1
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.] 4
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.] 3
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.] 5
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.] 3
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.] 6
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.] 1
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.] 7
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.] 2
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.] 8
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.] 6
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.] 9


/home/aryan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in power
  


In [40]:
print(ANN.confusion_matrix(x,y))

/home/aryan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in power
  


[[ 228 6755    0    0    0    0    0    0    0    0]
 [7931    0    0    0    0    0    0    0    0    0]
 [7173   18    0    0    0    0    0    0    0    0]
 [6889    7    0    0    0    0    0    0    0    0]
 [6831    9    0    0    0    0    0    0    0    0]
 [6378   22    0    0    0    0    0    0    0    0]
 [6762   37    0    0    0    0    0    0    0    0]
 [7275    5    0    0    0    0    0    0    0    0]
 [6853   47    0    0    0    0    0    0    0    0]
 [6777    3    0    0    0    0    0    0    0    0]]


In [23]:
print(y[0])

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
